In [29]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("../logs/vulhub_django_4.0.5.csv")

In [31]:
df.head()

,timestamp,container,pid,ppid,user,gid,comm,event_type,node,score,is_anomaly,phase
0,2025-09-30T21:28:01.568337+00:00,vulhub/django:4.0.5,28453,28415,root,0,bash,syscall_freq_agg,CASA,0.0000,0,warmup
1,2025-09-30T21:28:01.569216+00:00,vulhub/django:4.0.5,28453,28415,root,0,bash,syscall_freq_agg,CASA,0.1409,0,detection
2,2025-09-30T21:28:02.571540+00:00,vulhub/django:4.0.5,28536,28453,root,0,python,syscall_freq_agg,CASA,0.1115,0,detection
3,2025-09-30T21:28:11.127397+00:00,vulhub/django:4.0.5,28453,28415,root,0,bash,syscall_freq_agg,CASA,0.1380,0,detection
4,2025-09-30T21:28:11.128873+00:00,vulhub/django:4.0.5,28536,28453,root,0,python,syscall_freq_agg,CASA,0.1087,0,detection


In [32]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   100 non-null    object 
 1   container   100 non-null    object 
 2   pid         100 non-null    int64  
 3   ppid        100 non-null    int64  
 4   user        100 non-null    object 
 5   gid         100 non-null    int64  
 6   comm        100 non-null    object 
 7   event_type  100 non-null    object 
 8   node        100 non-null    object 
 9   score       100 non-null    float64
 10  is_anomaly  100 non-null    int64  
 11  phase       100 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 9.5+ KB


In [33]:
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)  # your strings look like ISO with offset
df = df.sort_values('timestamp').reset_index(drop=True)


In [34]:
attack_start = pd.to_datetime("2025-09-30T21:31:01+00:00", utc=True)
attack_end   = pd.to_datetime("2025-09-30T21:31:31+00:00", utc=True)


In [35]:
df['in_attack_window'] = (df['timestamp'] >= attack_start) & (df['timestamp'] <= attack_end)
df['is_anomaly'] = df['is_anomaly'].astype(int)


In [36]:
def compute_metrics(g):
    TP = ((g['is_anomaly'] == 1) & (g['in_attack_window'])).sum()
    FP = ((g['is_anomaly'] == 1) & (~g['in_attack_window'])).sum()
    FN = ((g['is_anomaly'] == 0) & (g['in_attack_window'])).sum()
    TN = ((g['is_anomaly'] == 0) & (~g['in_attack_window'])).sum()

    TPR = TP / (TP + FN) if (TP + FN) > 0 else np.nan
    FPR = FP / (FP + TN) if (FP + TN) > 0 else np.nan
    
    valid_detections = g.loc[(g['is_anomaly'] == 1) & (g['timestamp'] >= attack_end), 'timestamp']
    if not valid_detections.empty:
        first_detection = valid_detections.min()
        delay = (first_detection - attack_end).total_seconds()
    else:
        delay = np.nan
    return pd.Series({
        'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN,
        'TPR': TPR, 'FPR': FPR,
        'DetectionDelay(s)': delay
    })

In [37]:
event_metrics = df.groupby('event_type', group_keys=False).apply(compute_metrics)

print(event_metrics)

Series([], Name: timestamp, dtype: datetime64[ns, UTC])
90   2025-09-30 21:31:40.332666+00:00
91   2025-09-30 21:31:40.335387+00:00
94   2025-09-30 21:31:50.335884+00:00
99   2025-09-30 21:31:59.933798+00:00
Name: timestamp, dtype: datetime64[ns, UTC]
                   TP   FP   FN    TN  TPR       FPR  DetectionDelay(s)
event_type                                                             
resource          0.0  8.0  5.0  37.0  0.0  0.177778                NaN
syscall_freq_agg  0.0  4.0  5.0  41.0  0.0  0.088889           9.332666


/tmp/ipykernel_6412/3170710336.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  event_metrics = df.groupby('event_type', group_keys=False).apply(compute_metrics)


In [38]:
from datetime import datetime, timezone

# ISO 8601 string with 'Z'
iso_z_string = "2025-09-30T21:31:31.176Z"

# Parse the string into a datetime object
dt_object = datetime.fromisoformat(iso_z_string.replace('Z', '+00:00'))

# Format the datetime object with an explicit offset
explicit_offset_string = dt_object.astimezone(timezone.utc).isoformat(timespec='seconds')
# This will output '2025-10-01T14:30:00+00:00'

print(explicit_offset_string)

2025-09-30T21:31:31+00:00
